# LATIHAN LIVE CODE

## Introduction



    
     LIVE CODE  2
    =================================================
    Nama  : Muhammad Septian Zamzani
    Batch : hck8-021

    Program ini merupakan hasil pengujian dari Pembelajaran week 2 tentang sql dan  pandas
    =================================================


## Answer

##### Loading Data

In [1]:
from google.oauth2 import service_account

# Use a raw string (prefix with 'r') to avoid escape character issues
SERVICE_ACCOUNT_FILE_PATH = r"C:\Users\asus\Documents\iyan nitip\json\supple-gearbox-432504-c7-857effb9baf3.json"

# Specify the desired scopes (e.g., Gmail readonly)
SCOPES = ['https://www.googleapis.com/auth/bigquery']

# Create credentials from the service account file
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE_PATH, scopes=SCOPES)

from google.cloud import bigquery

# Initialize the client
client = bigquery.Client(credentials=credentials,
                         project=credentials.project_id,
                         default_query_job_config=bigquery.QueryJobConfig(default_dataset='ftds-hacktiv8-project.practice'))

In [2]:
# Kueri untuk mengambil data dari dataset BigQuery
df =client.query ('''
  SELECT *
  FROM ftds-hacktiv8-project.practice.bike_rent
  LIMIT 10
  ''').to_dataframe()
df.head()



,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes
0,18896589,Explorer,93,classic,2018-10-28 14:16:14+00:00,Dean Keeton & Speedway,11th & Salina,178
1,13948351,Local365,315,classic,2017-03-07 12:37:33+00:00,Main Office,Main Office,198
2,16243803,Local365,877,classic,2017-10-21 09:49:48+00:00,Republic Square @ 5th & Guadalupe,Main Office,60
3,16222446,Walk Up,273,classic,2017-10-18 15:01:52+00:00,Bullock Museum @ Congress & MLK,Main Office,60
4,16222451,Walk Up,201,classic,2017-10-18 15:02:26+00:00,Bullock Museum @ Congress & MLK,Main Office,60


### A.SQL

1. Berapa banyak label unique dari bike_type?

In [3]:
df =client.query ('''
  SELECT COUNT(DISTINCT bike_type) AS label_unique
  FROM bike_rent
  ''').to_dataframe()
df

,label_unique
0,2


2. Apa saja nama label unique dari tipe sepeda?

In [4]:
df =client.query ('''
  SELECT DISTINCT bike_type 
  FROM bike_rent
  ''').to_dataframe()
df

,bike_type
0,classic
1,electric


3. Berapa banyak total peminjaman sepeda pada tahun 2016 dan 2017?

In [5]:
df =client.query ('''
  SELECT COUNT(start_time) AS total_rent_in_2016_and_2017
  FROM bike_rent
  WHERE start_time > '2016-01-01' AND start_time < '2018-01-01'
  ''').to_dataframe()
df

,total_rent_in_2016_and_2017
0,7993


4. Dimana station yang paling banyak dikunjungi sebagai titik keberangkatan pada tahun 2016?

In [6]:
df =client.query ('''
  SELECT 
        start_station_name,
        COUNT(start_station_name) as best_station
  FROM bike_rent
  WHERE start_time > '2016-01-01' AND start_time < '2016-12-31'
  GROUP BY start_station_name
  ORDER BY best_station DESC
  LIMIT 1;
  ''').to_dataframe()
df

,start_station_name,best_station
0,Bullock Museum @ Congress & MLK,390


5. Berapa nilai minimum, maximum, dan rata-rata dari durasi perjalanan pada keseluruhan data?

In [7]:
df =client.query ('''
  SELECT 
        MAX(duration_minutes) AS max_duration_minutes,
        MIN(duration_minutes) AS min_duration_minutes,
        AVG(duration_minutes) AS mean_duration_minutes,
        
  FROM bike_rent
  
''').to_dataframe()
df

,max_duration_minutes,min_duration_minutes,mean_duration_minutes
0,2218,2,35.0995


6. Berapa rata-rata durasi perjalanan dari subscriber_type Explorer dan Weekender?

In [8]:
df =client.query('''
SELECT 
    --subscriber_type,
    AVG(duration_minutes) AS AVG_duration
FROM bike_rent
WHERE subscriber_type IN ('Explorer', 'Weekender')
--GROUP BY subscriber_type
                 
''').to_dataframe()
df

,AVG_duration
0,30.818905


7. Tampilkan entry data yang memiliki duration_time diatas 4.25 jam. (Gunakan subquery untuk menjawab pertanyaan ini dengan meng-convert duration_time dari menit ke hour).

In [17]:
df = client.query('''
SELECT *
FROM (
    SELECT *,
           ROUND(duration_minutes / 60,2) AS duration_hours
    FROM bike_rent
) AS subquery
WHERE duration_hours > 4.25
''').to_dataframe()
df

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes,duration_hours
0,13255686,Walk Up,539,classic,2016-12-25 11:17:16+00:00,Pease Park,Pease Park,351,5.85
1,13255691,Walk Up,453,classic,2016-12-25 11:18:17+00:00,Pease Park,Pease Park,350,5.83
2,14534164,Walk Up,664,classic,2017-04-21 20:46:17+00:00,Davis at Rainey Street,Pease Park,783,13.05
3,14534142,Walk Up,275,classic,2017-04-21 20:41:54+00:00,Davis at Rainey Street,Pease Park,789,13.15
4,11965934,Walk Up,453,classic,2016-09-17 11:55:23+00:00,Bullock Museum @ Congress & MLK,Lavaca & 6th,325,5.42
...,...,...,...,...,...,...,...,...,...
86,14699898,Walk Up,262,classic,2017-05-06 09:08:08+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,350,5.83
87,9516320,Walk Up,157,classic,2016-04-11 13:19:09+00:00,Lavaca & 6th,State Capitol @ 14th & Colorado,643,10.72
88,14465838,Walk Up,871,classic,2017-04-16 12:39:51+00:00,South Congress & Academy,State Capitol @ 14th & Colorado,684,11.40
89,15884138,Walk Up,882,classic,2017-09-07 19:11:26+00:00,Capital Metro HQ - East 5th at Broadway,Capital Metro HQ - East 5th at Broadway,292,4.87


### B.Pandas

1. Cek kesesuaian tipe data dan keberadaan missing values dengan .info()

In [9]:
df =client.query ('''
  SELECT *
  FROM ftds-hacktiv8-project.practice.bike_rent
  ''').to_dataframe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   trip_id             8000 non-null   Int64              
 1   subscriber_type     8000 non-null   object             
 2   bike_id             8000 non-null   object             
 3   bike_type           8000 non-null   object             
 4   start_time          8000 non-null   datetime64[us, UTC]
 5   start_station_name  8000 non-null   object             
 6   end_station_name    8000 non-null   object             
 7   duration_minutes    8000 non-null   Int64              
dtypes: Int64(2), datetime64[us, UTC](1), object(5)
memory usage: 515.8+ KB


2. Buat kolom baru bernama quarter.

Lakukan ekstraksi waktu (quarter dalam tahun) berdasarkan kolom start_time dengan ketentuan berikut:

Notes:

- Dalam menjawab soal Pandas no. 2, tidak boleh menggunakan ekstraksi waktu Series.dt.quarter
- Lakukan looping dan conditional dengan memanfaatkan ekstraksi bulan Series.dt.month terhadap kolom start_time, pastikan tipe datanya ialah datetime agar dapat dilakukan ekstraksi.
- Simpan hasil looping ke dalam suatu variabel, dan buat kolom baru berisikan variabel sebelumnya.

In [10]:
# df['quarter'] = df['start_time'].dt.quarter
# df.head()

data =[]
for i in df['start_time'].dt.month:
    if i == 1 or i == 2 or i == 3:
        data.append('Q1')
    if i == 4 or i == 5 or i == 6:
        data.append('Q2')
    if i == 7 or i == 8 or i == 9:
        data.append('Q3')
    if i == 10 or i == 11 or i == 12:
        data.append('Q1')
df['Quarter'] = data
df

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes,Quarter
0,18896589,Explorer,93,classic,2018-10-28 14:16:14+00:00,Dean Keeton & Speedway,11th & Salina,178,Q1
1,13948351,Local365,315,classic,2017-03-07 12:37:33+00:00,Main Office,Main Office,198,Q1
2,16243803,Local365,877,classic,2017-10-21 09:49:48+00:00,Republic Square @ 5th & Guadalupe,Main Office,60,Q1
3,16222446,Walk Up,273,classic,2017-10-18 15:01:52+00:00,Bullock Museum @ Congress & MLK,Main Office,60,Q1
4,16222451,Walk Up,201,classic,2017-10-18 15:02:26+00:00,Bullock Museum @ Congress & MLK,Main Office,60,Q1
...,...,...,...,...,...,...,...,...,...
7995,15469480,Walk Up,327,classic,2017-07-22 20:17:46+00:00,Bullock Museum @ Congress & MLK,Bullock Museum @ Congress & MLK,55,Q3
7996,16130650,Walk Up,528,classic,2017-10-07 11:25:36+00:00,Congress & Cesar Chavez,Bullock Museum @ Congress & MLK,55,Q1
7997,14204138,Walk Up,235,classic,2017-03-27 13:19:02+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55,Q1
7998,15037809,Walk Up,206,classic,2017-06-05 11:42:40+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55,Q2


3. Tampilkan data perjalanan dari tipe member Local365 dengan durasi perjalanan diatas 1 jam (Jika tidak ada entry yang memenuhi kondisi ini tidak apa, tuliskan kode pandasnya dengan lengkap saja).

In [11]:
trip_data = df[(df['subscriber_type'] == 'Local365') & (df['duration_minutes'] > 60)]
trip_data

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes,Quarter
1,13948351,Local365,315,classic,2017-03-07 12:37:33+00:00,Main Office,Main Office,198,Q1
94,15118837,Local365,693,classic,2017-06-14 12:48:06+00:00,Lavaca & 6th,Lavaca & 6th,81,Q2
165,12594741,Local365,332,classic,2016-10-28 11:13:44+00:00,Palmer Auditorium,Lavaca & 6th,147,Q1
359,14489810,Local365,955,classic,2017-04-18 08:02:59+00:00,11th & San Jacinto,11th & San Jacinto,134,Q2
401,18839331,Local365,343,classic,2018-10-19 19:43:22+00:00,Davis at Rainey Street,11th & San Jacinto,823,Q1
428,14041702,Local365,27,classic,2017-03-15 08:26:51+00:00,13th & San Antonio,13th & San Antonio,123,Q1
453,15862398,Local365,395,classic,2017-09-05 11:47:16+00:00,13th & San Antonio,13th & San Antonio,77,Q3
608,16520009,Local365,518,classic,2017-12-18 09:03:12+00:00,Palmer Auditorium,Congress & Cesar Chavez,290,Q1
609,15886237,Local365,862,classic,2017-09-08 08:07:12+00:00,South Congress & Academy,Congress & Cesar Chavez,269,Q3
704,9794912,Local365,30,classic,2016-05-01 10:50:37+00:00,South Congress & Academy,Barton Springs & Riverside,113,Q2


4. Berapa rata-rata durasi perjalanan dari tipe pelanggan Walk Up dengan tipe sepeda classic pada bulan Maret berdasarkan data yang diambil? (Jika tidak ada entry yang memenuhi kondisi ini tidak apa, tuliskan kode pandasnya dengan lengkap saja).


In [12]:
filtered_df = df[(df['subscriber_type'] == 'Walk Up') 
                 & (df['bike_type'] == 'classic') 
                 & (df['start_time'].dt.month == 3)]

# Menghitung rata-rata durasi perjalanan
average_duration = filtered_df['duration_minutes'].mean()

# print(f"Rata-rata durasi perjalanan adalah: {average_duration} menit")
average_duration

38.476584022038566

In [13]:
print(df.columns)

Index(['trip_id', 'subscriber_type', 'bike_id', 'bike_type', 'start_time',
       'start_station_name', 'end_station_name', 'duration_minutes',
       'Quarter'],
      dtype='object')
